# Data Preprocessing - heatmap

In [18]:
import pandas as pd
import numpy as np
import math
from datetime import datetime,timedelta
from scipy.stats import entropy

In [2]:
df = pd.read_csv('mc1-reports-data.csv')

In [3]:
df

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1
...,...,...,...,...,...,...,...,...
83065,2020-04-10 02:30:00,9.0,10.0,10.0,NaN,7.0,2.0,8
83066,2020-04-10 02:30:00,8.0,10.0,10.0,NaN,7.0,1.0,8
83067,2020-04-09 16:45:00,10.0,9.0,10.0,NaN,8.0,1.0,8
83068,2020-04-09 16:55:00,8.0,8.0,9.0,NaN,7.0,0.0,8


In [4]:
feature_list = ["sewer_and_water","power", "roads_and_bridges", "medical", "buildings","shake_intensity"]


In [5]:
for i in range(len(feature_list)):
    feature = feature_list[i]
    print("{} has NAN: {}".format(feature,df[feature].isnull().sum()))

sewer_and_water has NAN: 171
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 47441
buildings has NAN: 170
shake_intensity has NAN: 12144


# for Heatmap usage

In [6]:
regionGroup = df.groupby("location") #pandas.core.groupby.generic.DataFrameGroupBy object

In [154]:
for region_num in range(1,20):
    region_df = regionGroup.get_group(region_num)
    print('in region {}:'.format(region_num))
    for i in range(len(feature_list)):
        feature = feature_list[i]
        print("{} has NAN: {}".format(feature,region_df[feature].isnull().sum()))
    print('-----------------------')

in region 1:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 889
-----------------------
in region 2:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 5294
buildings has NAN: 0
shake_intensity has NAN: 974
-----------------------
in region 3:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 770
-----------------------
in region 4:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 2867
buildings has NAN: 0
shake_intensity has NAN: 225
-----------------------
in region 5:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 374
-----------------------
in region 6:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has N

In [ ]:
##create timelist: elements are timestamp
timelist = []
time_current = pd.to_datetime("2020-04-06 00:00:00")
time_end = pd.to_datetime("2020-04-11 00:01:00")
##(time_start - time_end).total_seconds() < 0
while (time_end -time_current).total_seconds() > 0:
    timelist.append(time_current)
    time_current += timedelta(minutes=30)

timelist_10hour = []
time_current = pd.to_datetime("2020-04-06 00:00:00")
time_end = pd.to_datetime("2020-04-11 00:01:00")
while (time_end - time_current).total_seconds() >= 0:
    timelist_10hour.append(time_current)
    time_current += timedelta(hours=10)
    
cols = ['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity','location'] 

region_dict = {(num+1): None for num in range(19)}

### To get data with interval as 10-hour, replace `convert_time_interval` with `convert_time_interval_10hour`, replace `timelist` with `timelist_10hour`

In [7]:
#function for coverting time to the last interval (08:00:00 -> 00:00:00)
def convert_time_interval_10hour(time):
    start_time = pd.to_datetime('2020-04-06 00:00:00')
    intervals = ((time - start_time).total_seconds()/3600)//10
    time = start_time + timedelta(hours=intervals*10)
    return time

#function for coverting time to the last interval (00:14 -> 00:00 00:59->00:30)
def convert_time_interval(time):
    approx = round(time.minute//30.0) * 30
    time = time.replace(minute=0)
    time += timedelta(seconds=approx * 60)
    return time

def convert_time_region(region_df):
    for idx, row in region_df.iterrows():
        current_time = row['time']
        time_interval = convert_time_interval(current_time)
        region_df.loc[idx,'time'] = time_interval
    return region_df

#function for sorting timeline of a region
def sort_regioni(i):
    """
    parameter: int i is referred to the region No.
    """
    region = regionGroup.get_group(i)
    region['time'] = pd.to_datetime(region.time)
    region = region.sort_values(by='time').reset_index().drop(labels=['index'], axis=1)
  ##  region = region.drop(['index'])
    region = convert_time_region(region)
    return region
    

In [8]:
for i in range(1,20):
    region_dict[i] = sort_regioni(i)

/Users/zhangyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
#checking used
region_dict[3]

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-06 00:00:00,6.0,3.0,10.0,10.0,9.0,NaN,3
1,2020-04-06 00:00:00,5.0,9.0,10.0,7.0,0.0,1.0,3
2,2020-04-06 00:00:00,4.0,7.0,5.0,0.0,9.0,NaN,3
3,2020-04-06 00:00:00,0.0,7.0,10.0,10.0,1.0,NaN,3
4,2020-04-06 00:00:00,3.0,3.0,5.0,4.0,4.0,NaN,3
...,...,...,...,...,...,...,...,...
13530,2020-04-10 23:00:00,6.0,10.0,3.0,1.0,1.0,NaN,3
13531,2020-04-10 23:30:00,8.0,9.0,4.0,7.0,4.0,NaN,3
13532,2020-04-10 23:30:00,3.0,0.0,1.0,10.0,3.0,NaN,3
13533,2020-04-10 23:30:00,4.0,2.0,1.0,5.0,5.0,1.0,3


In [10]:
heatmap_data_dict = {num: pd.DataFrame(columns=cols,index= timelist) for num in range(1,20)} ##each element should be a dataframe

for i in range(1,20):
    region_df = region_dict[i]
    current_df = heatmap_data_dict[i]
    timegroup_mean_df = region_df.groupby('time').mean()
    for index,row in timegroup_mean_df.iterrows():   ##intex now is timestamp object
        current_df.loc[index] = row
    current_df['location'] = current_df['location'].fillna((current_df['location'].mean()))
    current_df = current_df.fillna(-1).reset_index()
    heatmap_data_dict[i] = current_df

In [11]:
#checking used
heatmap_data_dict[1]

,index,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-06 00:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,1.0
1,2020-04-06 00:30:00,2.000000,10.000000,3.000000,5.000000,5.000000,-1.0,1.0
2,2020-04-06 01:00:00,7.500000,3.500000,6.000000,5.000000,4.500000,1.0,1.0
3,2020-04-06 01:30:00,9.000000,10.000000,2.000000,4.000000,0.000000,-1.0,1.0
4,2020-04-06 02:00:00,4.833333,6.166667,4.333333,8.333333,6.833333,-1.0,1.0
...,...,...,...,...,...,...,...,...
236,2020-04-10 22:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,1.0
237,2020-04-10 22:30:00,2.000000,9.000000,6.000000,7.500000,5.000000,1.0,1.0
238,2020-04-10 23:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,1.0
239,2020-04-10 23:30:00,4.000000,2.000000,3.000000,1.000000,7.000000,-1.0,1.0


In [137]:
merge_i_df = pd.concat([heatmap_data_dict[i] for i in range(1,20)])

In [138]:
def changename_i(string):
    if string == 'location' or string == 'index':
        return string
    else:
        return string+'_i'
merge_i_df.rename(mapper=changename_i, axis=1, inplace=True)

In [139]:
merge_i_df['location'] = merge_i_df['location'].astype(int)
merge_i_df

,index,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i,location
0,2020-04-06 00:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,1
1,2020-04-06 00:30:00,2.000000,10.000000,3.000000,5.000000,5.000000,-1.0,1
2,2020-04-06 01:00:00,7.500000,3.500000,6.000000,5.000000,4.500000,1.0,1
3,2020-04-06 01:30:00,9.000000,10.000000,2.000000,4.000000,0.000000,-1.0,1
4,2020-04-06 02:00:00,4.833333,6.166667,4.333333,8.333333,6.833333,-1.0,1
...,...,...,...,...,...,...,...,...
236,2020-04-10 22:00:00,7.000000,6.000000,0.000000,-1.000000,10.000000,-1.0,19
237,2020-04-10 22:30:00,10.000000,10.000000,8.000000,-1.000000,5.000000,-1.0,19
238,2020-04-10 23:00:00,3.333333,3.333333,8.333333,-1.000000,4.333333,1.0,19
239,2020-04-10 23:30:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,19


In [83]:
heatmap_data_dict_etp = {num: pd.DataFrame(columns=cols,index= timelist) for num in range(1,20)} 
##each element should be a dataframe

for i in range(1,20):
    region_df = region_dict[i]
    current_df = heatmap_data_dict_std[i]
    groupObject = region_df.groupby('time')
    for time in timelist:
        try:
            group_df = groupObject.get_group(time)
        except KeyError:
            continue
        else:
            ##do work here
            total = len(group_df)
            entropy_dict = {feature:-1 for feature in feature_list}
            poslist_dict = {feature:None for feature in feature_list}
            for col in cols:
                possibility_list = (group_df[col].value_counts()/total).tolist()
                poslist_dict[col] = possibility_list
                if possibility_list != []:
                    etp = entropy(poslist_dict[col], base=11)
                    entropy_dict[col] = etp
                heatmap_data_dict_etp[i].loc[time][col] = entropy_dict[col]
            heatmap_data_dict_etp[i].loc[time]['location'] = i
    heatmap_data_dict_etp[i][col] = heatmap_data_dict_etp[i][col].fillna(i)
    

In [84]:
#checking used
heatmap_data_dict_etp[1]

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
2020-04-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1
2020-04-06 00:30:00,0.0,0.0,0.0,0.0,0.0,-1,1
2020-04-06 01:00:00,0.289065,0.289065,0.0,0.289065,0.289065,0.0,1
2020-04-06 01:30:00,0.0,0.0,0.0,0.0,0.0,-1,1
2020-04-06 02:00:00,0.650867,0.650867,0.747222,0.650867,0.650867,-1,1
...,...,...,...,...,...,...,...
2020-04-10 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1
2020-04-10 22:30:00,0.289065,0.0,0.289065,0.289065,0.289065,0.0,1
2020-04-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1
2020-04-10 23:30:00,0.0,0.0,0.0,0.0,0.0,-1,1


In [140]:
def changename_u(string):
    if string == 'location' or string == 'index':
        return string
    else:
        return string+'_u'
merge_u_df = pd.concat([heatmap_data_dict_etp[i] for i in range(1,20)]).reset_index()
merge_u_df.rename(mapper=changename_u, axis=1, inplace=True)

In [141]:
merge_u_df = merge_u_df.fillna(-1)

In [142]:
merge_u_df

,index,sewer_and_water_u,power_u,roads_and_bridges_u,medical_u,buildings_u,shake_intensity_u,location
0,2020-04-06 00:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,1
1,2020-04-06 00:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0,1
2,2020-04-06 01:00:00,0.289065,0.289065,0.000000,0.289065,0.289065,0.0,1
3,2020-04-06 01:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0,1
4,2020-04-06 02:00:00,0.650867,0.650867,0.747222,0.650867,0.650867,-1.0,1
...,...,...,...,...,...,...,...,...
4574,2020-04-10 22:00:00,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.0,19
4575,2020-04-10 22:30:00,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.0,19
4576,2020-04-10 23:00:00,0.458157,0.265447,0.265447,-1.000000,0.458157,0.0,19
4577,2020-04-10 23:30:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,19


In [144]:
merge_i_df.set_index(['index','location'],inplace=True)
merge_u_df.set_index(['index','location'],inplace=True)

In [145]:
merge_u_df

,,sewer_and_water_u,power_u,roads_and_bridges_u,medical_u,buildings_u,shake_intensity_u
index,location,,,,,,
2020-04-06 00:00:00,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
2020-04-06 00:30:00,1,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
2020-04-06 01:00:00,1,0.289065,0.289065,0.000000,0.289065,0.289065,0.0
2020-04-06 01:30:00,1,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
2020-04-06 02:00:00,1,0.650867,0.650867,0.747222,0.650867,0.650867,-1.0
...,...,...,...,...,...,...,...
2020-04-10 22:00:00,19,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.0
2020-04-10 22:30:00,19,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.0
2020-04-10 23:00:00,19,0.458157,0.265447,0.265447,-1.000000,0.458157,0.0


In [146]:
merge_i_df

,,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i
index,location,,,,,,
2020-04-06 00:00:00,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
2020-04-06 00:30:00,1,2.000000,10.000000,3.000000,5.000000,5.000000,-1.0
2020-04-06 01:00:00,1,7.500000,3.500000,6.000000,5.000000,4.500000,1.0
2020-04-06 01:30:00,1,9.000000,10.000000,2.000000,4.000000,0.000000,-1.0
2020-04-06 02:00:00,1,4.833333,6.166667,4.333333,8.333333,6.833333,-1.0
...,...,...,...,...,...,...,...
2020-04-10 22:00:00,19,7.000000,6.000000,0.000000,-1.000000,10.000000,-1.0
2020-04-10 22:30:00,19,10.000000,10.000000,8.000000,-1.000000,5.000000,-1.0
2020-04-10 23:00:00,19,3.333333,3.333333,8.333333,-1.000000,4.333333,1.0


In [147]:
merge_df = merge_i_df.join(merge_u_df)

In [148]:
merge_df

,,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i,sewer_and_water_u,power_u,roads_and_bridges_u,medical_u,buildings_u,shake_intensity_u
index,location,,,,,,,,,,,,
2020-04-06 00:00:00,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
2020-04-06 00:30:00,1,2.000000,10.000000,3.000000,5.000000,5.000000,-1.0,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
2020-04-06 01:00:00,1,7.500000,3.500000,6.000000,5.000000,4.500000,1.0,0.289065,0.289065,0.000000,0.289065,0.289065,0.0
2020-04-06 01:30:00,1,9.000000,10.000000,2.000000,4.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
2020-04-06 02:00:00,1,4.833333,6.166667,4.333333,8.333333,6.833333,-1.0,0.650867,0.650867,0.747222,0.650867,0.650867,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-10 22:00:00,19,7.000000,6.000000,0.000000,-1.000000,10.000000,-1.0,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.0
2020-04-10 22:30:00,19,10.000000,10.000000,8.000000,-1.000000,5.000000,-1.0,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.0
2020-04-10 23:00:00,19,3.333333,3.333333,8.333333,-1.000000,4.333333,1.0,0.458157,0.265447,0.265447,-1.000000,0.458157,0.0


In [149]:
merge_df.to_csv('merge_ui.csv')